In [19]:
!echo $http_proxy

http://squid-compute101:3128


In [20]:
import comet_ml
comet_ml.login()

In [21]:
from comet_ml import Experiment

In [22]:
import keras
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.preprocessing import image
from keras.optimizers import RMSprop, Adam,SGD


from keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from keras.callbacks import Callback
import logging
import tensorflow as tf

In [23]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255


In [24]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [25]:
def build_Explod_grad(numfm, numnodes, input_shape=(28, 28, 1)):
    model = Sequential()

    # Input layer
    model.add(Conv2D(numfm, kernel_size = (5, 5), activation='relu', 
                     input_shape=input_shape))

    # Additional Conv2D layers
    model.add(Conv2D(64, (3, 3), activation='relu'))
    # Flattening layer
    model.add(Flatten())

    # Fully connected layers:
    model.add(Dense(numnodes, activation='relu'))

    # Output layer
    model.add(Dense(10, activation='softmax'))  # Output for 10-class classification

    return model

model = build_Explod_grad(numfm=64, numnodes=512, input_shape=(28, 28,1))

# Compile with high learning rate and no gradient control
model.compile(optimizer=SGD(learning_rate=8.0), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()










Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 24, 24, 64)     │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 22, 22, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 30976)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │    15,860,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,903,946 (60.67 MB)

 Trainable params: 15,903,946 (60.67 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
experiment = Experiment(project_name="Explode_grad")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/nast69/explode-grad/b576e4946b7e4fa2a782ef7a54d16a28

COMET INFO: Couldn't find a Git repository in '/lustre06/project/6078700/nast/comet' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


In [27]:
experiment.display()

In [28]:
class LoggingCallback(Callback):
    def __init__(self, experiment):
        self.experiment = experiment

    def on_epoch_end(self, epoch, logs=None):
        count = 0
        for weight in self.model.get_weights():
            name = "layer-" + str((count // 2) + 1) + "-" + ("bias" if count % 2 == 1 else "weight")
            self.experiment.log_histogram_3d(weight, name=name)
            count += 1
        

In [29]:
experiment.log_dataset_hash(x_train)

cb = LoggingCallback(experiment)
with experiment.train():
    model.fit(
        x_train,
        y_train,
        batch_size=32,
        epochs=50,
        callbacks=[cb],
        validation_data=(x_test, y_test)
    )

# callbacks=[cb],
score = model.evaluate(x_test, y_test, verbose=0)
logging.info("Score %s", score)
experiment.end()

Epoch 1/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.0997 - loss: 224921569747718373376.0000 - val_accuracy: 0.0980 - val_loss: 2.3704
Epoch 2/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0981 - loss: 2.4106 - val_accuracy: 0.0980 - val_loss: 2.4869
Epoch 3/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.1020 - loss: 2.4053 - val_accuracy: 0.1032 - val_loss: 2.3442
Epoch 4/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.1015 - loss: 2.4078 - val_accuracy: 0.1009 - val_loss: 2.4765
Epoch 5/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.1025 - loss: 2.4030 - val_accuracy: 0.0982 - val_loss: 2.3321
Epoch 6/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.1020 - loss: 2.3982 - val_accuracy: 0.0974 - val_loss: 2.4156
Epoch 7/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.0987 - loss: 2.4115 - val_accuracy: 0.1010 - val_loss: 2.4024
Epoch 8/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.10

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : separate_mouse_1965
COMET INFO:     url                   : https://www.comet.com/nast69/explode-grad/b576e4946b7e4fa2a782ef7a54d16a28
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     train_accuracy [50]                  : (0.0986500009894371, 0.1034500002861023)
COMET INFO:     train_batch_accuracy [9400]          : (0.0, 0.21875)
COMET INFO:     train_batch_loss [9400]              : (2.174854040145874, 6.360314408345226e+21)
COMET INFO:     train_epoch_duration [50]            : (3.359103557013441, 5.142564114998095)
COMET INFO:     train_loss [50]                      : (2.399935483932495, 3.73138446368574e+19)
COMET INFO:     train_val_